In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl

from terminal_optimization import hydrogen_objects
from terminal_optimization import hydrogen_defaults
from terminal_optimization import hydrogen_system

# 0. Prepare inputs

In [2]:
startyear = 2018
lifecycle = 10

## 0.1 Generate demand forecast scenario

In [3]:
years = list(range(startyear, startyear+lifecycle))

demand = []
for year in years:
    if year < 2023:
        demand.append(1_000_000)
    else:
        demand.append(5_000_000)

scenario_data={'year': years, 'volume': demand}

# instantiate a commodity objects
lhydrogen = hydrogen_objects.Commodity(**hydrogen_defaults.commodity_lhydrogen_data)
lhydrogen.scenario_data = pd.DataFrame(data=scenario_data)

# Ammonia
hydrogen_defaults.commodity_ammonia_data['historic_data'] = []
hydrogen_defaults.commodity_ammonia_data['scenario_data'] = []
ammonia = hydrogen_objects.Commodity(**hydrogen_defaults.commodity_ammonia_data)

# MCH
hydrogen_defaults.commodity_MCH_data['historic_data'] = []
hydrogen_defaults.commodity_MCH_data['scenario_data'] = []
MCH = hydrogen_objects.Commodity(**hydrogen_defaults.commodity_MCH_data)

# combine
demand = [lhydrogen, ammonia, MCH]

# plot scenarios 
if False:
    lhydrogen.plot_demand()
    ammonia.plot_demand()
    MCH.plot_demand()

## 0.2 Generate vessels

In [4]:
# instantiate vessels
Smallhydrogen = hydrogen_objects.Vessel(**hydrogen_defaults.smallhydrogen_data)
Largehydrogen = hydrogen_objects.Vessel(**hydrogen_defaults.largehydrogen_data)
Smallammonia = hydrogen_objects.Vessel(**hydrogen_defaults.smallammonia_data)
Largeammonia = hydrogen_objects.Vessel(**hydrogen_defaults.largeammonia_data)
Handysize = hydrogen_objects.Vessel(**hydrogen_defaults.handysize_data)
Panamax = hydrogen_objects.Vessel(**hydrogen_defaults.panamax_data)
VLCC = hydrogen_objects.Vessel(**hydrogen_defaults.vlcc_data)


vessels = [Smallhydrogen, Largehydrogen, Smallammonia, Largeammonia, Handysize, Panamax, VLCC] 

# 1. Instatiate terminal system

In [5]:
Terminal = hydrogen_system.System(startyear = startyear, 
                         lifecycle = lifecycle, 
                         elements = demand + vessels, 
                         operational_hours = 5840,   # example Wijnand
                         debug=True,
                         storage_type_defaults=hydrogen_defaults.storage_lh2_data)

# 2. Start simulation
Develop terminal given the throughput scenario and investment triggers implemented

In [6]:
Terminal.simulate()


Simulate year: 2018
  Total vessel calls: 0
     Small Hydrogen  calls: 0
     Large Hydrogen calls: 0
     Small ammonia calls: 0
     Large ammonia calls: 0
     Handysize calls: 0
     Panamax calls: 0
     VLCC calls: 0
  Total cargo volume: 0.0
     a total of 0 [] is online; 0 total planned
     a total of 0 [] is online; 0 total planned
     a total of 0 [] is online; 0 total planned
     a total of 0 [] is online; 0 total planned
     a total of 0 [] is online; 0 total planned
     a total of 0 [] is online; 0 total planned
     a total of 0 [] is online; 0 total planned

  Start analysis:
     Berth occupancy planned (@ start of year): inf
     Berth occupancy online (@ start of year): inf
     Unloading occupancy planned (@ start of year): inf
     Unloading occupancy online (@ start of year): inf
     waiting time factor (@ start of year): inf
     waiting time occupancy (@ start of year): nan
     throughput online 0
     throughput planned 0
  *** add Berth to elements
  

c:\users\stephanie lanphen\documents\terminal-optimization\src\terminal_optimization\hydrogen_system.py:1154: RuntimeWarning: invalid value encountered in double_scalars
  waiting_time_occupancy = waiting_time_hours * total_calls / self.operational_hours



Simulate year: 2020
  Total vessel calls: 54
     Small Hydrogen  calls: 30
     Large Hydrogen calls: 24
     Small ammonia calls: 0
     Large ammonia calls: 0
     Handysize calls: 0
     Panamax calls: 0
     VLCC calls: 0
  Total cargo volume: 1000000.0
     a total of 1 Berth_01 is online; 1 total planned
     a total of 1 Jetty_01 is online; 1 total planned
     a total of 1 jetty_pipeline_01 is online; 1 total planned
     a total of 11 HTank_01 is online; 11 total planned
     a total of 3 H2retrieval_LH2_01 is online; 3 total planned
     a total of 1 hinterland_pipeline_01 is online; 1 total planned
     a total of 1 Hinterland_station_01 is online; 1 total planned

  Start analysis:
     Berth occupancy planned (@ start of year): 0.1202054794520548
     Berth occupancy online (@ start of year): 0.1202054794520548
     Unloading occupancy planned (@ start of year): 0.09246575342465753
     Unloading occupancy online (@ start of year): 0.09246575342465753
     waiting time f

     a total of 96000 ton of HydrogenTank storage capacity is online; 144000 ton total planned
  *** add storage to elements
     a total of 96000 ton of HydrogenTank storage capacity is online; 148000 ton total planned
  *** add storage to elements
     a total of 96000 ton of HydrogenTank storage capacity is online; 152000 ton total planned
  *** add storage to elements
     a total of 96000 ton of HydrogenTank storage capacity is online; 156000 ton total planned
  *** add storage to elements
     a total of 96000 ton of HydrogenTank storage capacity is online; 160000 ton total planned
  *** add storage to elements
     a total of 96000 ton of HydrogenTank storage capacity is online; 164000 ton total planned
  *** add storage to elements
     a total of 96000 ton of HydrogenTank storage capacity is online; 168000 ton total planned
  *** add storage to elements
     a total of 96000 ton of HydrogenTank storage capacity is online; 172000 ton total planned
  *** add storage to elements


NameError: name 'commodity_type_defaults' is not defined

## Report all elements

In [ ]:
if False: 
    for element in Terminal.elements:
        print("")
        print(element.name)
        print("")
        print(element.__dict__)

In [ ]:
Terminal.terminal_elements_plot()
Terminal.demand_terminal_plot()

In [ ]:
if True: 
    # to plot the terminal occupany's 
    Terminal.terminal_occupancy_plot() #occupancy of the berth 
    Terminal.plant_occupancy_plot() #occupancy of the plant
    Terminal.station_occupancy_plot() #occupancy of the loading station
    
    # to plot the capacities of the different elements 
    Terminal.Jetty_capacity_plot() #jetty
    Terminal.Pipeline1_capacity_plot() #pipeline from jetty to storage
    Terminal.Storage_capacity_plot() #storage
    Terminal.H2retrieval_capacity_plot() # H2 retrieval
    Terminal.Pipeline2_capacity_plot() #pipeline from H2 retrieval to hinterland        

In [ ]:
# add cash flow information for each of the Terminal elements
cash_flows, cash_flows_WACC_real = Terminal.add_cashflow_elements()

In [ ]:
Terminal.cashflow_plot(cash_flows)

In [ ]:
Terminal.cashflow_plot(cash_flows_WACC_real)

## SandBox

In [ ]:
Terminal.NPV()